In [ ]:
# set working directory
get_ipython().magic(u'pwd')
get_ipython().magic(u'cd "C:/Users/xxxxx"')

import os
import glob
from glob import glob as gg

from collections import defaultdict

import re
import sqlite3

import pandas as pd
from pandas import DataFrame
from pandas import Series

import numpy as np
from numpy import random, asarray, linspace, corrcoef
from datetime import datetime, date, time

import jieba
import jieba.posseg as pseg
import jieba.analyse
from optparse import OptionParser
from __future__ import print_function, unicode_literals
import sys

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',300)

#load SQlite database
con = sqlite3.connect("OccupyHK.sqlite")
cur = con.cursor()
df = pd.read_sql_query("SELECT * FROM Posts WHERE published_date BETWEEN '2014-12-01' AND '2014-12-31'", con)
con.close()

# using the example of Occupy Central Facebook pages
df_yellow_official = df.loc[df['org_name'].isin(['學民思潮 Scholarism',
                                                 'Occupy Central 佔領中環', 
                                                 '讓愛與和平佔領中環 Occupy Central with Love and Peace', 
                                                 '926 平民在政總現場','佔領期間 民間智慧選輯', '傘下爸媽',
                                                 '香港天主教正義和平委員會','Hong Kong Democracy Now', '踢爆五毛','香港警權關注組',
                                                 '維護香港核心價值', '我係香港人', '香港人 Secrets','保衛香港自由聯盟','9反佔中 HK oic',
                                                 'Fluid Occupiers'])] 

df_yellow_all = df.loc[df['feed_id'].isin([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])] 
df_yellow_nonofficial = df_yellow_all[~df_yellow_all.org_name.isin(['學民思潮 Scholarism', 
                                                                    'Occupy Central 佔領中環', 
                                                                    '讓愛與和平佔領中環 Occupy Central with Love and Peace', 
                                                                    '926 平民在政總現場','佔領期間 民間智慧選輯', '傘下爸媽',
                                                                    '香港天主教正義和平委員會','Hong Kong Democracy Now', '踢爆五毛','香港警權關注組',
                                                                    '維護香港核心價值', '我係香港人', '香港人 Secrets','保衛香港自由聯盟',
                                                                    '9反佔中 HK oic','Fluid Occupiers'])]

df_blue_official = df.loc[df['org_name'].isin(['幫港出聲 Silent Majority','Speak Out HK 港人講地','藍絲帶運動 The Blue Ribbon Movement','向香港警察致敬',
                                               '反佔中苦主大聯盟','泛民不代表我','聲討黃之鋒',
                                               '一人一Like救學生，反對中學生衝擊警察及佔中！',
                                               '「保普選 反佔中」簽左名揮手區！',
                                               '我要真普洱行動','我撐警務處長曾偉雄！',
                                               '我係大學生，我愛國愛港'])] 


df_blue_all = df.loc[df['feed_id'].isin([20,21,22,23,24,25,26,27,28,29,30,31])] 

df_blue_nonofficial = df_blue_all[~df_blue_all.org_name.isin(['幫港出聲 Silent Majority','Speak Out HK 港人講地','藍絲帶運動 The Blue Ribbon Movement','向香港警察致敬',
                                               '反佔中苦主大聯盟','泛民不代表我','聲討黃之鋒',
                                               '一人一Like救學生，反對中學生衝擊警察及佔中！',
                                               '「保普選 反佔中」簽左名揮手區！',
                                               '我要真普洱行動','我撐警務處長曾偉雄！',
                                               '我係大學生，我愛國愛港'])]

print ("There are",len(df_yellow_official), "OFFICIAL posts sent by anti-establishment civil groups")
print ("There are",len(df_yellow_nonofficial), "UNOFFICIAL posts sent by anti-establishment civil groups")
print ("There are",len(df_yellow_all), "posts sent by anti-establishment civil groups")
print ("There are",len(df_blue_official), "OFFICIAL posts sent by pro-establishment civil groups")
print ("There are",len(df_blue_nonofficial), "UNOFFICIAL posts sent by pro-establishment civil groups")
print ("There are",len(df_blue_all), "posts sent by pro-establishment civil groups")

In [ ]:
#convert to corpus
def convert2corpus(df):
    corpus = []
    for index, i in df.iterrows():
        corpus.append(i['content'])
    corpus = str(corpus)
    corpus = corpus.replace("[" and "]", " ")
    corpus = corpus.replace("\\", " ")
    return corpus

processed_corpus = convert2corpus(df_yellow_official) 
print ("the length of the processed corpus is:",len(processed_corpus))

In [ ]:
def cnpuncttoen(corpus):
    #corpus_new = corpus.replace(".", "")
    corpus_new = corpus.replace("。", "*.")
    corpus_new = corpus_new.replace("？", "*?")
    corpus_new = corpus_new.replace(",", "*,")
    corpus_new = corpus_new.replace("！", "*!")
    return corpus_new
    
processed_corpus_clean = cnpuncttoen(processed_corpus) 
#processed_corpus_clean[500]

In [ ]:
# segment Chinese sentences see http://blog.ourren.com/2015/02/21/nltk_note_token_sentence/

#tokenize into sentences
def sentence_split(str_centence):
    list_ret = list()
    for s_str in str_centence.split('*.'):
        if '*?' in s_str:
            list_ret.extend(s_str.split('*?'))
        elif '*!' in s_str:
            list_ret.extend(s_str.split('*!'))
        else:
            list_ret.append(s_str)
    return list_ret

processed_corpus_sent = sentence_split(processed_corpus_clean)
processed_corpus_sent[20]

In [ ]:
# POS Tagging
# apply stop word list
jieba.analyse.set_stop_words('C:/Users/xxxxx/cn_stopword.txt')

pos_corpus_sent = [[(j.word, j.flag) for j in jieba.posseg.cut(i)] for i in processed_corpus_sent]
#pos_corpus_sent[113]

In [ ]:
# select only nouns and adjectives
matched_pos_corpus_sent = [[token for token, tag in sent if re.match('n|a', tag) and len(token)>1] 
                for sent in pos_corpus_sent]

print ("the length of matched corpus containing nouns and adjectives is:", len(matched_pos_corpus_sent))
#matched_pos_corpus_sent[113]

In [ ]:
# create a semantic network with nouns and adjectives as nodes. More refer to http://www.sociology-hacks.org/?p=151

import itertools as it
edgelist = [edge for phrase in matched_pos_corpus_sent for edge in it.combinations(phrase, 2)]   

import networkx as nx
word_network = nx.Graph(edgelist)
word_network_centrality = nx.betweenness_centrality(word_network, normalized=True)
sorted_centrality = sorted(word_network_centrality.items(), key=lambda x:x[1], reverse=True)
for word, centr in sorted_centrality[:30]:
        print (word)
        print (centr)

print (nx.number_of_edges(word_network))
print (nx.number_of_nodes(word_network))

nx.write_graphml(word_network, "word_network_official_yellow_2014_12.graphml")

In [ ]:
# get simple resonance score to compare similarity of two texts

def simple_resonance(index_a, index_b):
    sorted_a = sorted(index_a.items(), key=lambda x:x[1], reverse=True)
    sorted_b = sorted(index_b.items(), key=lambda x:x[1], reverse=True)
    scores = [a[1]*b[1] for a, b in zip(sorted_a, sorted_b) if a[0] == b[0]]
    return sum(scores)

simple_resonance(word_network_centrality, word_network_centrality_2)

In [ ]:
#get standardized resonance score to compare similarity of two texts

import math
def standardized_sr(index_a, index_b):
    sum_a_squared = sum([centr**2 for word, centr in index_a.items()])
    sum_b_squared = sum([centr**2 for word, centr in index_b.items()])
    norm = math.sqrt((sum_a_squared * sum_b_squared))
    standardized = simple_resonance(index_a, index_b) / norm
    return standardized

standardized_sr(word_network_centrality, word_network_centrality_2)